In [3]:
import numpy as np
from pathlib import Path
import mne
from util import *
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt   
import pandas as pd
from sklearn.metrics import cohen_kappa_score

### Data Processing

In [20]:
n_sample = 72
k = 6
bin_size = int(n_sample / k)
#n_channel, n_time, n_sample = EEGR_train[0][0].shape
randIdx = np.random.permutation(n_sample)
randIdx = randIdx.reshape(k, bin_size)

In [5]:
passbands = [
    (1, 3), (2, 5), (4, 7), (6, 10), (7, 12),
    (10, 15), (12, 19), (15, 25), (19, 30),
    (25, 35), (30, 40)
]

gdf_folder = Path("BCICIV_2a_gdf")

target_events = {'769': 'left', '770': 'right', '771': 'feet', '772': 'tongue'}

filtered_data = []
labels_2d = []

# skipping AO4T because only has eye movement
users = ["A01T.gdf", "A02T.gdf", "A03T.gdf", "A05T.gdf", "A06T.gdf", "A07T.gdf", "A08T.gdf", "A09T.gdf"]

for subject in users:
    # Iterate through users
    gdf_path = gdf_folder / subject
    print(f"\nProcessing {subject}...")
    
    raw = mne.io.read_raw_gdf(gdf_path.resolve(), preload=True)
    events, event_id_map = mne.events_from_annotations(raw)

    subject_data = []
    user_labels = []

    user_event_ids = {target_events[key]: event_id_map[key] for key in target_events if key in event_id_map}

    # Filter and epoch for each band
    for band in passbands:
        low, high = band
        print(f"  Filtering {low}-{high} Hz...")

        # Bandpass filter
        raw_filtered = raw.copy().filter(l_freq=low, h_freq=high, method='fir', fir_design='firwin')

        # Create epochs
        epochs = mne.Epochs(raw_filtered, events, event_id=user_event_ids, baseline=(None, 0), event_repeated='merge', preload=True, tmin=-.3, tmax=4)
        labels = epochs.events[:, -1]  # Last column contains the event ID

        # Collect data and labels
        subject_data.append(epochs.get_data()[:, :-3, :])  # Shape: (n_trials, n_channels, n_times)
        user_labels.append(labels)

    filtered_data.append(subject_data)
    labels_2d.append(user_labels)


Processing A01T.gdf...
Extracting EDF parameters from F:\COGS189-BCI-Competition-Project\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


f:\Anaconda\envs\CSE156\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
  Filtering 1-3 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 3.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Filter length: 825 samples (3.300 s)

Not setting metadata
288 matching events found


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 2-5 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 5.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Filter length: 413 samples (1.652 s)

Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events a

...

0 bad epochs dropped
  Filtering 25-35 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 133 samples (0.532 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 30-40 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 111 samples (0.444 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped


**Separating data into 4 groups**

In [6]:
data_left = []
data_right = []
data_feet = []
data_tongue = []

for subject in range(len(filtered_data)):
    subject_data_left = []
    subject_data_right = []
    subject_data_feet = []
    subject_data_tongue = []
    for band in range(len(filtered_data[subject])):
        band_data_left = []
        band_data_right = []
        band_data_feet = []
        band_data_tongue = []
        for label in range(len(labels_2d[subject][band])):
            if labels_2d[subject][band][label] == 7:
                band_data_left.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 8:
                band_data_right.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 9:
                band_data_feet.append(filtered_data[subject][band][label])
            else:
                band_data_tongue.append(filtered_data[subject][band][label])
        subject_data_left.append(band_data_left)
        subject_data_right.append(band_data_right)
        subject_data_feet.append(band_data_feet)
        subject_data_tongue.append(band_data_tongue)
    data_left.append(subject_data_left)
    data_right.append(subject_data_right)
    data_feet.append(subject_data_feet)
    data_tongue.append(subject_data_tongue)

data_left = np.array(data_left)
data_right = np.array(data_right)
data_feet = np.array(data_feet)
data_tongue = np.array(data_tongue)

data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076))

In [7]:
data_OVR_left = []
data_OVR_right = []
data_OVR_feet = []
data_OVR_tongue = []

random_samples = np.random.permutation(72)
random_sample_range1 = random_samples[:24]
random_sample_range2 = random_samples[24:48]
random_sample_range3 = random_samples[48:72]

# data_OVR_left
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_left.append(subject_data)

# data_OVR_right
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_right.append(subject_data)

# data_OVR_feet
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_feet.append(subject_data)

# data_OVR_tongue
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_feet[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_tongue.append(subject_data)

# Convert to numpy arrays
data_OVR_left = np.array(data_OVR_left)
data_OVR_right = np.array(data_OVR_right)
data_OVR_feet = np.array(data_OVR_feet)
data_OVR_tongue = np.array(data_OVR_tongue)

In [8]:
# # Reordering dimensions
data_left = data_left.transpose(0, 1, 3, 4, 2)
data_right = data_right.transpose(0, 1, 3, 4, 2)
data_feet = data_feet.transpose(0, 1, 3, 4, 2)
data_tongue = data_tongue.transpose(0, 1, 3, 4, 2)

data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72))

In [9]:
# # Reordering dimensions
data_OVR_left = data_OVR_left.transpose(0, 1, 3, 4, 2)
data_OVR_right = data_OVR_right.transpose(0, 1, 3, 4, 2)
data_OVR_feet = data_OVR_feet.transpose(0, 1, 3, 4, 2)
data_OVR_tongue = data_OVR_tongue.transpose(0, 1, 3, 4, 2)

data_OVR_left.shape, data_OVR_right.shape, data_OVR_feet.shape, data_OVR_tongue.shape

((8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72))

## Shannon Entropy

In [10]:
import numpy as np

def shannon_entropy(signal_data, num_bins=10):
    signal_data = signal_data.flatten()  # Flatten trials and time
    signal_data = (signal_data - np.min(signal_data)) / (np.max(signal_data) - np.min(signal_data) + 1e-8)  # Normalize

    hist, _ = np.histogram(signal_data, bins=num_bins, density=True)
    p = hist / np.sum(hist)  # Normalize to get probabilities
    p = p[p > 0]  # Remove zero probabilities

    return -np.sum(p * np.log2(p))  # Compute Shannon entropy

def compute_entropy_top_channels(data, class_name, channels, top_n=5):
    num_subjects, num_bands, num_channels, _, num_trials = data.shape
    entropy_per_channel = {ch: [] for ch in channels}

    for subject in range(num_subjects):
        for band in range(num_bands):
            trials = np.array(data[subject][band])  # Extract (channels, time, trials)
            trials = np.moveaxis(trials, -1, 0)  # Reshape to (trials, channels, time)

            for ch_idx, (ch_name, ch_values) in enumerate(entropy_per_channel.items()):
                entropy = shannon_entropy(trials[:, ch_idx, :])  # Calculate entropy for each channel
                entropy_per_channel[ch_name].append(entropy)

    # Compute average entropy across subjects and bands
    avg_entropy_per_channel = {ch: np.mean(ent) for ch, ent in entropy_per_channel.items()}

    # Sort channels by entropy in descending order
    sorted_entropy = sorted(avg_entropy_per_channel.items(), key=lambda x: x[1], reverse=True)

    return sorted_entropy  # Return top N channels

# Compute top channels based on entropy
eeg_channels = raw.info["ch_names"][:22] # we have removed eog channels
top_channels_left = compute_entropy_top_channels(data_left, "Left Hand", channels=eeg_channels)
top_channels_right = compute_entropy_top_channels(data_right, "Right Hand", channels=eeg_channels)
top_channels_feet = compute_entropy_top_channels(data_feet, "Feet", channels=eeg_channels)
top_channels_tongue = compute_entropy_top_channels(data_tongue, "Tongue", channels=eeg_channels)

In [11]:
# find the subset of channels that are most discriminative
top_n = 10
unique_channels = set()
for class_name, sorted_entropy in zip(["Left hand", "Right hand", "Feet", "Tongue"], 
                                      [top_channels_left, top_channels_right, top_channels_feet, top_channels_tongue]):
    print(f"\nTop {top_n} channels for {class_name} motor imagery:")
    for ch, ent in sorted_entropy[:top_n]:
        print(f"{ch}: {ent:.4f}")
        unique_channels.add(ch)

print(unique_channels)


Top 10 channels for Left hand motor imagery:
EEG-Cz: 1.9903
EEG-7: 1.9781
EEG-C4: 1.9777
EEG-6: 1.9776
EEG-C3: 1.9761
EEG-11: 1.9668
EEG-12: 1.9578
EEG-3: 1.9574
EEG-10: 1.9552
EEG-13: 1.9523

Top 10 channels for Right hand motor imagery:
EEG-Cz: 1.9888
EEG-11: 1.9752
EEG-7: 1.9749
EEG-12: 1.9729
EEG-6: 1.9726
EEG-10: 1.9572
EEG-13: 1.9563
EEG-C4: 1.9536
EEG-15: 1.9519
EEG-16: 1.9511

Top 10 channels for Feet motor imagery:
EEG-7: 2.0313
EEG-C4: 2.0306
EEG-6: 2.0259
EEG-Cz: 2.0251
EEG-C3: 2.0054
EEG-12: 2.0047
EEG-10: 2.0026
EEG-11: 2.0016
EEG-3: 1.9999
EEG-2: 1.9948

Top 10 channels for Tongue motor imagery:
EEG-C4: 1.9976
EEG-C3: 1.9939
EEG-7: 1.9860
EEG-Cz: 1.9827
EEG-13: 1.9751
EEG-6: 1.9700
EEG-9: 1.9656
EEG-2: 1.9648
EEG-12: 1.9587
EEG-11: 1.9578
{'EEG-C3', 'EEG-C4', 'EEG-Cz', 'EEG-12', 'EEG-10', 'EEG-6', 'EEG-13', 'EEG-16', 'EEG-9', 'EEG-11', 'EEG-2', 'EEG-15', 'EEG-7', 'EEG-3'}


In [12]:
selected_channels = []
for ch in unique_channels:
    for idx in range(len(eeg_channels)):
        if eeg_channels[idx] == ch:
            selected_channels.append(idx)
selected_channels = sorted(selected_channels)
selected_channels = np.array(selected_channels)
selected_channels, len(selected_channels)

(array([ 3,  4,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 20, 21]), 14)

## Test Data

In [13]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline

import moabb
from moabb.datasets import BNCI2014_001
from moabb.evaluations import WithinSessionEvaluation
from moabb.paradigms import LeftRightImagery


moabb.set_log_level("info")
warnings.filterwarnings("ignore")

dataset = BNCI2014_001()

In [14]:
sessions = dataset.get_data()

In [15]:
passbands = [
    (1, 3), (2, 5), (4, 7), (6, 10), (7, 12),
    (10, 15), (12, 19), (15, 25), (19, 30),
    (25, 35), (30, 40)
]

filtered_data = []
labels_2d = []

# skipping AO4T because only has eye movement
runs = ["0", "1", "2", "3", "4", "5"]

skip = 4

for subject in range(1, 10):
    print("Subject: ", subject)
    if subject == skip:
        continue

    # List to store all raw objects
    raw_list = []
    
    # Iterate over runs and add each raw to the list
    for run in runs:
        raw = sessions[subject]["1test"][run]
        raw_list.append(raw)
    
    # Concatenate all raw objects into one
    combined_raw = mne.concatenate_raws(raw_list)
    
    # Iterate through users
    
    raw = combined_raw
    events, event_id = mne.events_from_annotations(raw)

    subject_data = []
    user_labels = []

    # Filter and epoch for each band
    for band in passbands:
        low, high = band

        # Bandpass filter
        raw_filtered = raw.copy().filter(l_freq=low, h_freq=high, method='fir', fir_design='firwin')

        # Create epochs
        epochs = mne.Epochs(raw_filtered, events, event_id=event_id, baseline=(None, 0), event_repeated='merge', preload=True, tmin=-.3, tmax=4)
        labels = epochs.events[:, -1]  # Last column contains the event ID

        # Collect data and labels
        subject_data.append(epochs.get_data()[:, :-1, :])  # Shape: (n_trials, n_channels, n_times)
        user_labels.append(labels)
    filtered_data.append(subject_data)
    labels_2d.append(user_labels)

Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9


In [16]:
test_data_left = []
test_data_right = []
test_data_feet = []
test_data_tongue = []

for subject in range(len(filtered_data)):
    subject_data_left = []
    subject_data_right = []
    subject_data_feet = []
    subject_data_tongue = []
    for band in range(len(filtered_data[subject])):
        band_data_left = []
        band_data_right = []
        band_data_feet = []
        band_data_tongue = []
        for label in range(len(labels_2d[subject][band])):
            if labels_2d[subject][band][label] == 2:
                band_data_left.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 3:
                band_data_right.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 1:
                band_data_feet.append(filtered_data[subject][band][label])
            else:
                band_data_tongue.append(filtered_data[subject][band][label])
        subject_data_left.append(band_data_left)
        subject_data_right.append(band_data_right)
        subject_data_feet.append(band_data_feet)
        subject_data_tongue.append(band_data_tongue)
    test_data_left.append(subject_data_left)
    test_data_right.append(subject_data_right)
    test_data_feet.append(subject_data_feet)
    test_data_tongue.append(subject_data_tongue)

test_data_left = np.array(test_data_left)
test_data_right = np.array(test_data_right)
test_data_feet = np.array(test_data_feet)
test_data_tongue = np.array(test_data_tongue)

test_data_left.shape, test_data_right.shape, test_data_feet.shape, test_data_tongue.shape

((8, 11, 72, 25, 1076),
 (8, 11, 72, 25, 1076),
 (8, 11, 72, 25, 1076),
 (8, 11, 72, 25, 1076))

In [17]:
test_data_OVR_left = []
test_data_OVR_right = []
test_data_OVR_feet = []
test_data_OVR_tongue = []

random_samples = np.random.permutation(72)
random_sample_range1 = random_samples[:24]
random_sample_range2 = random_samples[24:48]
random_sample_range3 = random_samples[48:72]

# data_OVR_left
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_right[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_left.append(subject_data)

# data_OVR_right
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_right.append(subject_data)

# data_OVR_feet
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_feet.append(subject_data)

# data_OVR_tongue
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(test_data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(test_data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(test_data_feet[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    test_data_OVR_tongue.append(subject_data)

# Convert to numpy arrays
test_data_OVR_left = np.array(test_data_OVR_left)
test_data_OVR_right = np.array(test_data_OVR_right)
test_data_OVR_feet = np.array(test_data_OVR_feet)
test_data_OVR_tongue = np.array(test_data_OVR_tongue)

test_data_OVR_left = test_data_OVR_left.transpose(0, 1, 3, 4, 2)
test_data_OVR_right = test_data_OVR_right.transpose(0, 1, 3, 4, 2)
test_data_OVR_feet = test_data_OVR_feet.transpose(0, 1, 3, 4, 2)
test_data_OVR_tongue = test_data_OVR_tongue.transpose(0, 1, 3, 4, 2)

test_data_left = test_data_left.transpose(0, 1, 3, 4, 2)
test_data_right = test_data_right.transpose(0, 1, 3, 4, 2)
test_data_feet = test_data_feet.transpose(0, 1, 3, 4, 2)
test_data_tongue = test_data_tongue.transpose(0, 1, 3, 4, 2)

In [18]:
test_data_left.shape

(8, 11, 25, 1076, 72)

## Left vs Rest - Test Data

In [21]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
test_size = 72
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)
lvr = []

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(8):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        band_score_test = np.zeros((test_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            l_train = data_left[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_left[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            l_val = data_left[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_left[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            l_test = test_data_left[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            OVR_test = test_data_OVR_left[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((l_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((l_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)
            test_data = np.stack((l_test,OVR_test), axis=0) # <- CSP input data format (classes, trials, chans, samples)


            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            test_prob = test(np.vstack(test_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))
            y_true = np.append(np.ones(72), np.zeros(72))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
            band_score_test[:,band] = test_prob

        
        
        y_pred = np.rint(band_score_test.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_true)/len(y_true)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_true)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))
lvo = np.mean(kappa, axis=1).tolist()

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Right vs Rest - Test Data

In [22]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
test_size = 72
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(8):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        band_score_test = np.zeros((test_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            r_train = data_right[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_right[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            r_val = data_right[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_right[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            r_test = test_data_right[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            OVR_test = test_data_OVR_right[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((r_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((r_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)
            test_data = np.stack((r_test,OVR_test), axis=0) # <- CSP input data format (classes, trials, chans, samples)


            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            test_prob = test(np.vstack(test_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))
            y_true = np.append(np.ones(72), np.zeros(72))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
            band_score_test[:,band] = test_prob

        
        
        y_pred = np.rint(band_score_test.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_true)/len(y_true)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_true)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))
rvo = np.mean(kappa, axis=1).tolist()

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Feet vs Rest - Test Data

In [23]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
test_size = 72
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(8):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        band_score_test = np.zeros((test_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            f_train = data_feet[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_feet[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            f_val = data_feet[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_feet[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            f_test = test_data_feet[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            OVR_test = test_data_OVR_feet[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((f_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((f_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)
            test_data = np.stack((f_test,OVR_test), axis=0) # <- CSP input data format (classes, trials, chans, samples)


            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            test_prob = test(np.vstack(test_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))
            y_true = np.append(np.ones(72), np.zeros(72))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
            band_score_test[:,band] = test_prob

        
        
        y_pred = np.rint(band_score_test.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_true)/len(y_true)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_true)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))
fvo = np.mean(kappa, axis=1).tolist()

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Tongue vs Rest - Test Data

In [24]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
test_size = 72
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(8):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        band_score_test = np.zeros((test_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            t_train = data_tongue[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_tongue[subject, band][selected_channels][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            t_val = data_tongue[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_tongue[subject, band][selected_channels][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            t_test = test_data_tongue[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            OVR_test = test_data_OVR_tongue[subject, band][selected_channels][:,:,:].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((t_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((t_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)
            test_data = np.stack((t_test,OVR_test), axis=0) # <- CSP input data format (classes, trials, chans, samples)


            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            test_prob = test(np.vstack(test_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))
            y_true = np.append(np.ones(72), np.zeros(72))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
            band_score_test[:,band] = test_prob

        
        
        y_pred = np.rint(band_score_test.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_true)/len(y_true)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_true)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))
tvo = np.mean(kappa, axis=1).tolist()

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

In [25]:
#kappa values
avgKappa = {}
for i in range(len(lvo)):
    subject = i+1
    avg = (lvo[i] + rvo[i] + fvo[i] + tvo[i])/4
    if subject >= 4:
        subject += 1
    avgKappa[f"Subject {subject}"] = avg
    
avgKappa

{'Subject 1': 0.806712962962963,
 'Subject 2': 0.45949074074074076,
 'Subject 3': 0.7627314814814814,
 'Subject 5': 0.3234953703703704,
 'Subject 6': 0.47858796296296297,
 'Subject 7': 0.8055555555555556,
 'Subject 8': 0.5040509259259259,
 'Subject 9': 0.8032407407407407}

In [26]:
selected_channels

array([ 3,  4,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 20, 21])